# Neural Networks + Cloud GPU Lab
Today we'll be introducing how to use processing accelerators, called Graphics Processing Units (GPUs), to greatly speed up the training of neural networks.

## Background
In the olden days of machine learning, CPUs, like the Intel one that's probably in your laptop right now, were used to train neural networks. The problem was, these CPUs were too slow, and as a result it took weeks to train fairly small networks. If you've tried training a network on your laptop, you've probably experienced this.

In recent years, however, the advent of Graphics Processing Units (GPUs) in deep learning has greatly accelerated network training speed by multiple orders of magnitude. GPUs are used for processing computer graphics in games, which so happens makes their design very useful for doing lots of linear algebra math! Nowadays a GPU can reduce the training time from weeks/months down to minutes, depending on the network and dataset used.

### AWS and Google Colaboratory
Buying your own GPU is of course, expensive, so big tech companies have made them available to rent (or use for free in the case of Google Colab) on the internet. Amazon Web Services (AWS), is the most popular provider of GPUs in the "cloud" (online rented compute power). They provide remote access to Linux machines in their datacenters, and we'll be going over how to use these to train models in the cloud. We'll also be going over how to use Google Colaboratory, which is a free service by Google that allows you to use Jupyter notebooks in your web browser in conjunction with (FREE!) GPUs.

## Google Colaboratory Introduction
Google Colab is located at [Google's website](https://colab.research.google.com/notebooks/welcome.ipynb). In order to create a new notebook, you can either import an existing notebook or create your own. Today we'll be importing this notebook, so go ahead and upload this notebook using File -> Upload Notebook.

![](https://github.com/mlberkeley/NMEP-sp19/blob/master/labs/week4/colab_upload.PNG?raw=true)

You can see it looks just like a Jupyter notebook, but it's in Google! Now we're going to go ahead and enable GPU support. Click edit -> Notebook Settings -> change Hardware Accelerator to GPU. Now you have access to a (free!) K80 GPU for training purposes. Because Google offers this service for free, you'll be limited to 12 hours of contiguous use before your instance is killed, and your instance can be killed if Google runs low on resources. Also, you're limited to TensorFlow only (Keras accessible through `tf.keras`), so no PyTorch or custom code.

You can now proceed to run whatever operations you want just like you would in a normal Jupyter notebook. For now though, we're going to pause to go over how to use a cloud GPU in AWS to train larger models.

## Amazon Web Services (AWS) Introduction

Head to [AWS](https://aws.amazon.com/) and make an account if you don't already have one. This part will cost you a slight amount of money (<$1 depending on how long you take), so make sure you have a legitimate credit card linked to your AWS account.

After you've created your account, go to the [Account Dashboard](https://us-west-2.console.aws.amazon.com/console/home?region=us-west-2#) and under the Services dropdown menu, click EC2. AWS EC2 is Amazon's name for its rentable online compute servers. Click the Launch Instance button to begin launching a new instance.

![Launch Instance](https://github.com/mlberkeley/NMEP-sp19/blob/master/labs/week4/aws_ec2.PNG?raw=true)

You should then be presented with a list of Amazon Machine Images (AMIs). AMIs are the operating system that will be installed on your cloud machine. Use the search bar to find the `Deep Learning AMI (Ubuntu)` AMI, make sure the x86 option is checked, and click select.

You should now be presented with the Choose an Instance Type page (below). AWS uses its own internal names to represent the type of machine you'll be renting (you can look up the details on AWS' pages), and we'll be using their cheapest GPU instance, the `p2.xlarge`. This instance contains 1 K80 GPU (~5 years old as of 2019) and should suffice for our purposes, although you may choose their newer and significantly faster instances for larger project work in the future.
![Instance Type](https://github.com/mlberkeley/NMEP-sp19/blob/master/labs/week4/p2_xlarge.PNG?raw=true)

Go ahead and select the p2.xlarge instance and click Next. Make sure you don't click on Review and Launch!

The next page, Configure Instance, can be skipped. The fourth step, Configure Storage, can be skipped for this tutorial, but you should add storage as necessary to accomodate whatever datasets you need for your projects. The Add Tags step can also be skipped.

**The next step is very important!**

On the Configure Security Group step, there should be only a single entry, SSH listed. Change the Source column of this row to "Anywhere". This will allow you to remotely access the machine from any IP address or computer as long as you have the machine's key.
![Security Group](https://github.com/mlberkeley/NMEP-sp19/blob/master/labs/week4/security_group.PNG?raw=true)

Now click Review, and click Launch. You will be presented with the dialog below asking you to create a private key. Give the private key a name and download your newly created key.
![Private Key](https://github.com/mlberkeley/NMEP-sp19/blob/master/labs/week4/private_key.PNG?raw=true)

**IT IS VERY IMPORTANT THAT YOU DO NOT 1) LOSE THIS FILE 2) UPLOAD IT TO ANY PUBLIC AREA (INCLUDING GIT/HUB!) DOING SO WILL EXPOSE ACCESS TO YOUR INSTANCE TO ANYONE WHO WANTS TO GET INTO IT!**

The private key of your instance is the file that allows your computer (or anyone else who has the key) to connect to your cloud machine.

Click Launch.

### SSH into the AWS instance
Open a new command line and navigate to the directory you saved your private key in. Run the command below to SSH (connect to) your newly created instance. Your AWS instance public IP address can be found in your AWS EC2 dashboard.

`ssh -i [PRIVATE_KEY_NAME_INCLUDING_PEM_EXT] ubuntu@[PUBLIC IP HERE]`

You should see your command line open up with some Ubuntu terms and conditions and the current directory go to `/`.

### Anaconda Virtual Environments
In order to avoid dependencies from different projects conflicting with each other due to Python's global namespace, we're going to create something called a virtual environment. This essentially isolates all the dependencies we're going to use (tensorflow, etc) to only be active when we're using this virutal environment. Fortunately, a software called Anaconda (preinstalled on our AMI) takes care of most of the work for us.

To create a new virtual environment, run the following w/ package distro `tensorflow-gpu`:

`conda create -n [ENV NAME HERE] python=[PYTHON VERSION HERE, USUALLY 3.6] [PACKAGE DISTRO HERE, CAN LEAVE EMPTY]`

To then activate the virtual environment, run the following:

`source activate [ENV NAME]`

To deactivate your virtual environment and restore the default Python global environment, run the following:

`source deactivate`

### Training ResNet-20 on CIFAR-10
Inside the `python` folder of this lab you will find a file called `run.py`. Activate your Tensorflow-GPU virtual environment and run the file to see the magic of GPUs :)

You may need to install `numpy`, `keras`, and `h5py` if those aren't installed automatically. Run `pip install [packages here]` to do so.

#### What's going on here?
The script we just ran initializes a deep Convolutional Neural Network model named ResNet that was state of the art for classifying images (ie whether this picture is a fish, elephant, grass, etc) in 2015. This model has a high number of parameters (and it's a convolutional network), so it takes quite a while to train on a CPU. Fortunately, our GPU is able to easily train the high number of parameters this model has. In this case, we're training it from scratch to classify images on CIFAR-10, a set of 60000 32x32 RGB images that fit into 10 different categories.

**Caveat: the script uses the test set as a validation set so you can see how your test accuracy improves over time. Please don't do this in real life.**

Like watching paint dry? Run the script on your laptop :)

You're done with the lab once you've achieved 80% test accuracy. We use a much shallower and older model (20 layers) in this lab than what is commonly used in research/industry (50-200 layers + lots of fancy augmentation) to make things much faster for us. We also don't train for hours/days at a time - usually we'd train for 100+ epochs.